In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import re
import unicodedata
import os
import codecs
import nltk

from nltk.corpus import *
from collections import *
from gensim import *
from gensim.models import *
from sklearn import *
from sklearn.metrics.pairwise import *
from nltk.stem import *



metadatadf = pd.read_csv('metadata.csv', skipinitialspace=True)
reviewsdf = pd.read_csv('reviews.csv', skipinitialspace=True)

dictionary = corpora.Dictionary.load('reviewsDict.dict')
corpus = corpora.MmCorpus('reviewsDict.mm')
wem = Word2Vec.load('models/wem/wem.wem')
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

/Users/greenexplosions/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/greenexplosions/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/greenexplosions/anaconda/lib/python2.7/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This mod

In [2]:
documents = [str(reviewsdf.loc[i].review) for i in range(len(reviewsdf[:]))]

In [3]:
def generate_models(topic_num):
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=topic_num)
    lda = models.LdaModel(corpus, id2word=dictionary, num_topics=topic_num)
    return {'lsi':lsi,'lda':lda}

In [4]:
def analogy(model,a,b,c):
    return model.wv.most_similar(positive=[c, b], negative=[a])

In [5]:
analogy(wem,'rock','punk','metal')

[('math', 0.6342626810073853),
 ('grindcore', 0.592850387096405),
 ('hardcore', 0.5893312692642212),
 ('stoner', 0.5795284509658813),
 ('thrash', 0.5732924938201904),
 ('sabbath', 0.5669950246810913),
 ('metals', 0.564648449420929),
 ('punks', 0.5396698713302612),
 ('sludge', 0.5383000373840332),
 ('post-hardcore', 0.5367418527603149)]

In [6]:
def get_nice_topics(model):
    num_topics = len(model.get_topics())
    return str(num_topics)+'\n'+('\n\n'.join([model.print_topic(i) for i in list(range(num_topics))]))

models_list = [generate_models(i) for i in list(range(5,40))]
lsafile = open('models/lsa.txt', 'w')
ldafile = open('models/lda.txt', 'w')
for i in list(range(len(models_list))):
    lsafile.write(get_nice_topics(models_list[i]['lsi']))
    ldafile.write(get_nice_topics(models_list[i]['lda']))
    lsafile.write('\n\n')
    ldafile.write('\n\n')
lsafile.close()
ldafile.close()
for i in list(range(len(models_list))):
    (models_list[i]['lsi']).save('models/lsa/'+str(i)+'.lsi')
    (models_list[i]['lda']).save('models/lda/'+str(i)+'.lda')